In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Webscraping

In [20]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

df = pd.DataFrame(columns=["date", "text", "link"])
debatelink = []

base_url = "https://www.debates.org"
page = requests.get(urljoin(base_url, "/voter-education/debate-transcripts/"))
soup = BeautifulSoup(page.content, 'html.parser')

links = soup.find_all("a")
for link in links:
    if "debate-transcript" in link.get("href") and "Media" not in link.get("href"):
        debatelink.append(urljoin(base_url, link.get("href")))
        df.loc[len(df)] = [None, None, urljoin(base_url, link.get("href"))]

for link in df["link"]:
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        text = soup.find_all("p")
        text = [t.get_text() for t in text]
        text = " ".join(text)
        df.loc[df["link"] == link, "text"] = text

        date_link = link.split("/")[5:6] 
        date_link = "-".join(date_link[0].split("-")[:3])
        df.loc[df["link"] == link, "date"] = date_link
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {link}: {e}")

df


,date,text,link
0,,Unofficial transcripts of most presidential an...,https://www.debates.org/voter-education/debate...
1,vice-presidential-debate,Vice Presidential Debate at the University of ...,https://www.debates.org/voter-education/debate...
2,october-22-2020,Presidential Debate at Belmont University in N...,https://www.debates.org/voter-education/debate...
3,september-29-2020,Presidential Debate at Case Western Reserve Un...,https://www.debates.org/voter-education/debate...
4,october-19-2016,Presidential Debate at the University of Nevad...,https://www.debates.org/voter-education/debate...
...,...,...,...
92,october-22-1976,"\nOctober 22, 1976\n The Third Carter-Ford Pre...",https://www.debates.org/voter-education/debate...
93,september-26-1960,"\nSeptember 26, 1960\n The First Kennedy-Nixon...",https://www.debates.org/voter-education/debate...
94,october-7-1960,"\nOctober 7, 1960\n The Second Kennedy-Nixon P...",https://www.debates.org/voter-education/debate...
95,october-13-1960,"\nOctober 13, 1960\n The Third Kennedy-Nixon P...",https://www.debates.org/voter-education/debate...


In [22]:
df['text'][1]

'Vice Presidential Debate at the University of Utah in Salt Lake City, Utah October 07, 2020 PARTICIPANTS:\nSenator Kamala Harris (D-CA) and\nVice President Mike Pence (R) MODERATOR:\nSusan Page (USA Today) PAGE:\xa0Good evening. From the University of Utah in Salt Lake City, welcome to the first, and only, vice presidential debate of 2020, sponsored by the nonpartisan Commission on Presidential Debates. I’m Susan Page of USA TODAY. It is my honor to moderate this debate, an important part of our democracy. In Kingsbury Hall tonight we have a small and socially distant audience and we’ve taken extra precautions during this pandemic. Among other things, everyone in the audience is required to wear a face mask and the candidates will be seated 12 feet apart. The audience is enthusiastic about their candidates, but they’ve agreed to express that enthusiasm, only twice. At the end of the debate and now when I introduce the candidates. Please welcome California Senator Kamala Harris and Vic

In [17]:
len(debatelink)

97

In [7]:
req = requests.get('https://www.presidency.ucsb.edu/documents/republican-candidates-debate-des-moines-iowa-1')

In [8]:
req = requests.get('https://www.presidency.ucsb.edu/documents/republican-candidates-debate-des-moines-iowa-1')

In [9]:
req.status_code

200

In [10]:
req.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"\n  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">\n<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#">\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta charset="utf-8"><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"dee899de70",applicationID:"80106271"};;/*! For license information please see nr-loader-rum-1.250.0.min.js.LICENSE.txt */\n(()=>{var e,t,r={234:(e,t,r)=>{"use strict";r.d(t,{P_:()=>v,Mt:()=>b,C5:()=>s,DL:()=>A,OP:()=>S,lF:()=>T,Yu:()=>x,Dg:()=>m,CX:()=>c,GE:()=>_,sU:()=>R});var n=r(8632),i=r(

In [11]:
req.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"\n  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">\n<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#">\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta charset="utf-8"><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"dee899de70",applicationID:"80106271"};;/*! For license information please see nr-loader-rum-1.250.0.min.js.LICENSE.txt */\n(()=>{var e,t,r={234:(e,t,r)=>{"use strict";r.d(t,{P_:()=>v,Mt:()=>b,C5:()=>s,DL:()=>A,OP:()=>S,lF:()=>T,Yu:()=>x,Dg:()=>m,CX:()=>c,GE:()=>_,sU:()=>R});var n=r(8632),i=r(9

In [12]:
sitesoup = BeautifulSoup(req.content)

In [16]:
print(search)

[]


In [26]:
[tag.text for tag in sitesoup.select('p')]

['About\xa0Search',
 '',
 'PARTICIPANTS:Governor Ron DeSantis (FL); andFormer Governor Nikki Haley (SC);',
 'MODERATORS:Dana Bash (CNN); andJake Tapper (CNN)',
 'TAPPER: Live from Drake University in Des Moines, Iowa, this is the CNN Republican presidential debate.',
 'We want to welcome our viewers in the United States and around the world watching us on CNN, CNN Max, CNN International, CNN en Espanol, and CNN.com.',
 "Good evening. I'm Jake Tapper, moderating tonight's debate alongside my friend and colleague Dana Bash.",
 'BASH: And just five days from now, voters here in Iowa will cast the first votes of the 2024 presidential race.',
 "Tonight, the very first one-on-one matchup between top contenders Governor Ron DeSantis and former Governor and U.N. Ambassador Nikki Haley. Former President Trump met the requirements for tonight's debate, but declined to participate.",
 'TAPPER: Our commitment in this debate is to provide as much information as possible to voters, especially here i

In [35]:
import re
for elem in sitesoup(text=re.compile('TAPPER')):
    child = find_next_siblings(elem) 
    print(child)

NameError: name 'find_next_siblings' is not defined

In [3]:
presidency = pd.read_json('../../presidency.json')

In [6]:
presidency.head(30)

,Abraham Lincoln,Andrew Jackson,Andrew Johnson,Barack Obama,Benjamin Harrison,Calvin Coolidge,Chester A. Arthur,Dwight D. Eisenhower,Franklin D. Roosevelt,Franklin Pierce,...,Theodore Roosevelt,Thomas Jefferson,Ulysses S. Grant,Warren G. Harding,William Henry Harrison,William Howard Taft,William J. Clinton,William McKinley,Woodrow Wilson,Zachary Taylor
Debates,[],[],[],"[{'category': 'General Election', 'document_da...",[],[],[],[],[],[],...,[],[],[],[],[],[],"[{'category': 'General Election', 'document_da...",[],[],[]
E.O.P.,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
OMB,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
Oral,"[{'category': 'Address', 'document_date': '186...","[{'category': 'Address', 'document_date': '182...","[{'category': 'Address', 'document_date': '186...","[{'category': 'Address', 'document_date': '200...","[{'category': 'Address', 'document_date': '188...","[{'category': 'Address', 'document_date': '192...","[{'category': 'Address', 'document_date': '188...","[{'category': 'Address', 'document_date': '195...","[{'category': 'Address', 'document_date': '193...","[{'category': 'Address', 'document_date': '185...",...,"[{'category': 'Address', 'document_date': '190...","[{'category': 'Address', 'document_date': '180...","[{'category': 'Address', 'document_date': '186...","[{'category': 'Address', 'document_date': '192...","[{'category': 'Address', 'document_date': '184...","[{'category': 'Address', 'document_date': '190...","[{'category': 'Address', 'document_date': '199...","[{'category': 'Address', 'document_date': '189...","[{'category': 'Address', 'document_date': '191...","[{'category': 'Address', 'document_date': '184..."
Party Platforms,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
Written,"[{'category': 'Messages', 'document_date': '18...","[{'category': 'Messages', 'document_date': '18...","[{'category': 'Messages', 'document_date': '18...","[{'category': 'Messages', 'document_date': '20...","[{'category': 'Messages', 'document_date': '18...","[{'category': 'Messages', 'document_date': '19...","[{'category': 'Messages', 'document_date': '18...","[{'category': 'Messages', 'document_date': '19...","[{'category': 'Messages', 'document_date': '19...","[{'category': 'Messages', 'document_date': '18...",...,"[{'category': 'Messages', 'document_date': '19...","[{'category': 'Messages', 'document_date': '18...","[{'category': 'Messages', 'document_date': '18...","[{'category': 'Executive Orders', 'document_da...","[{'category': 'Messages', 'document_date': '18...","[{'category': 'Messages', 'document_date': '19...","[{'category': 'Messages', 'document_date': '19...","[{'category': 'Messages', 'document_date': '18...","[{'category': 'Messages', 'document_date': '19...","[{'category': 'Messages', 'document_date': '18..."


# NLP

In [3]:
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import string
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, ConfusionMatrixDisplay

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shelleywang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/shelleywang/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shelleywang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocessing(text, tokenizer, stopwords, lemmatizer):
    # Lower case
    text = text.lower()
    
    # Tokenize
    tokens = tokenizer.tokenize(text)
    
    # Remove stopwords
    tokens =  [token for token in tokens if token not in stopwords]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

In [ ]:
tokenized_df = df['text'].apply((lambda x: preprocessing(x, tokenizer, stopwords, lemmatizer)))
df['tokens'] = tokenized_df

In [ ]:
df['string_tokens'] = df['tokens'].str.join(" ")

## NLP Modelling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["string_tokens"], df['target'], random_state=42)

In [ ]:
vectorizer = TfidfVectorizer()
vectorizr.fit(X_train
X_train_vectorized = vectorizer.transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
mnb = MultinomialNB()
mnb = mnb.fit(X_train_vectorized, y_train)
